# Пробы

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from dataclasses import dataclass
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from numpy.typing import NDArray
from PIL import Image
from rich.pretty import pprint as pp
from transformers.modeling_utils import load_state_dict
from torchmetrics.functional.pairwise import pairwise_cosine_similarity

from libs.lizi.my_magi import MyMagiModel
from libs.lizi.my_magi.config import MagiConfig
from libs.lizi.my_magi.utils import read_image_as_np_array as read_image
from libs.lizi.my_magi.utils import UnionFind

ImportError: /home/userr/MANGA/detection/.pixi/envs/default/lib/python3.11/site-packages/torch/lib/libtorch_cuda.so: undefined symbol: cudaLaunchKernelExC, version libcudart.so.11.0

In [ ]:
torch.cuda.is_available()

True

## Загружаем страницы

In [ ]:
images = [read_image(str(image)) for image in Path("data/manga3/").glob("*.jpg")]

## Инициализируем модель

Загружаем pre-train веса из файла в dict

In [ ]:
state_dict = load_state_dict(str(Path("models/magi/pytorch_model.bin").resolve()))
state_dict.keys()

dict_keys(['ocr_model.encoder.embeddings.cls_token', 'ocr_model.encoder.embeddings.position_embeddings', 'ocr_model.encoder.embeddings.patch_embeddings.projection.weight', 'ocr_model.encoder.embeddings.patch_embeddings.projection.bias', 'ocr_model.encoder.encoder.layer.0.attention.attention.query.weight', 'ocr_model.encoder.encoder.layer.0.attention.attention.key.weight', 'ocr_model.encoder.encoder.layer.0.attention.attention.value.weight', 'ocr_model.encoder.encoder.layer.0.attention.output.dense.weight', 'ocr_model.encoder.encoder.layer.0.attention.output.dense.bias', 'ocr_model.encoder.encoder.layer.0.intermediate.dense.weight', 'ocr_model.encoder.encoder.layer.0.intermediate.dense.bias', 'ocr_model.encoder.encoder.layer.0.output.dense.weight', 'ocr_model.encoder.encoder.layer.0.output.dense.bias', 'ocr_model.encoder.encoder.layer.0.layernorm_before.weight', 'ocr_model.encoder.encoder.layer.0.layernorm_before.bias', 'ocr_model.encoder.encoder.layer.0.layernorm_after.weight', 'ocr_mo

Считываем конфиг из локальной директории и инициализируем нашу модель с ним

In [ ]:
config: MagiConfig = MagiConfig.from_json_file(Path("libs/lizi/my_magi/config.json").resolve())  # type: ignore
model = MyMagiModel(config)

Загружаем pre-train веса в модель

In [ ]:
model.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['ocr_model.encoder.embeddings.cls_token', 'ocr_model.encoder.embeddings.position_embeddings', 'ocr_model.encoder.embeddings.patch_embeddings.projection.weight', 'ocr_model.encoder.embeddings.patch_embeddings.projection.bias', 'ocr_model.encoder.encoder.layer.0.attention.attention.query.weight', 'ocr_model.encoder.encoder.layer.0.attention.attention.key.weight', 'ocr_model.encoder.encoder.layer.0.attention.attention.value.weight', 'ocr_model.encoder.encoder.layer.0.attention.output.dense.weight', 'ocr_model.encoder.encoder.layer.0.attention.output.dense.bias', 'ocr_model.encoder.encoder.layer.0.intermediate.dense.weight', 'ocr_model.encoder.encoder.layer.0.intermediate.dense.bias', 'ocr_model.encoder.encoder.layer.0.output.dense.weight', 'ocr_model.encoder.encoder.layer.0.output.dense.bias', 'ocr_model.encoder.encoder.layer.0.layernorm_before.weight', 'ocr_model.encoder.encoder.layer.0.layernorm_before.bias', 'ocr_model.encoder.encoder

In [ ]:
model.cuda() # type: ignore

MyMagiModel(
  (crop_embedding_model): ViTMAEModel(
    (embeddings): ViTMAEEmbeddings(
      (patch_embeddings): ViTMAEPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
    )
    (encoder): ViTMAEEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTMAELayer(
          (attention): ViTMAEAttention(
            (attention): ViTMAESelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTMAEIntermediate(
            (dense): Linear(in_features=768, out_features=

## Inference

image_bboxes - list со вложенными lists, вложенный список - одна страница, где каждый элемент - np.array
character_scores - list с тензорами, тензор - одна страница, каждый элемент - для каждого bbox score
crop_embeddings_for_batch - list с тензорами, где каждый тензор - страница, а строка - для каждого bbox
crop_bboxes - list c тензорами, где каждый тензор - страница, а строка - для каждого bbox

In [ ]:
from dataclasses import dataclass, asdict

In [ ]:
with torch.no_grad():
    crop_bboxes, crop_embeddings_for_batch, image_bboxes, character_scores = model.get_crops_and_embeddings(images)

/home/userr/micromamba/envs/manga311/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608853085/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [ ]:
character_scores[0]

In [ ]:
type(crop_embeddings_for_batch)

In [ ]:
for i in range(len(crop_embeddings_for_batch)):
    if i == 0:
        combined_tensor = crop_embeddings_for_batch[i]
    else:
        combined_tensor = torch.cat((combined_tensor, crop_embeddings_for_batch[i]), dim=0)

In [ ]:
combined_tensor.size()

In [ ]:
first_page = combined_tensor.cuda()
first_page.shape

torch.Size([16, 768])

In [ ]:
combined_tensor.size()

In [ ]:
combined_tensor[7].unsqueeze(0).size()

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
cos_matrix_np = cos_matrix.cpu().numpy()

In [ ]:
first_page_np = first_page.cpu().numpy()

In [12]:
cos_matrix =  pairwise_cosine_similarity(first_page, first_page)
cos_matrix

tensor([[1.0000, 0.9297, 0.9122, 0.4220, 0.3838, 0.4331, 0.5009, 0.7580, 0.6124,
         0.4703, 0.7207, 0.7264, 0.5305, 0.7301, 0.3716, 0.5543],
        [0.9297, 1.0000, 0.9484, 0.4216, 0.3583, 0.4454, 0.4737, 0.7653, 0.4817,
         0.4450, 0.7854, 0.6862, 0.5430, 0.7090, 0.3610, 0.5662],
        [0.9122, 0.9484, 1.0000, 0.4098, 0.3382, 0.4261, 0.4617, 0.7895, 0.4611,
         0.4283, 0.7880, 0.7045, 0.5158, 0.7065, 0.3551, 0.5482],
        [0.4220, 0.4216, 0.4098, 1.0000, 0.5134, 0.9528, 0.5269, 0.4975, 0.4503,
         0.9116, 0.4256, 0.6700, 0.7774, 0.6608, 0.7918, 0.7940],
        [0.3838, 0.3583, 0.3382, 0.5134, 1.0000, 0.5071, 0.9334, 0.5439, 0.5823,
         0.6558, 0.5002, 0.6100, 0.7967, 0.5711, 0.5820, 0.7580],
        [0.4331, 0.4454, 0.4261, 0.9528, 0.5071, 1.0000, 0.5278, 0.4523, 0.4083,
         0.9298, 0.3881, 0.6247, 0.7664, 0.6204, 0.7541, 0.7821],
        [0.5009, 0.4737, 0.4617, 0.5269, 0.9334, 0.5278, 1.0000, 0.6235, 0.7013,
         0.6565, 0.6223, 0.7158, 0.78

In [ ]:
# Применяем DBSCAN с косинусной схожестью
db = DBSCAN(metric='precomputed', eps=0.5, min_samples=5).fit(matrix)

# Проверяем метки кластеров
labels = db.labels_
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
first_character = torch.cat((combined_tensor[12],combined_tensor[7]), dim=0)

In [ ]:
first_character = torch.cat((combined_tensor[0:3],combined_tensor[7].unsqueeze(0)), dim=0)

In [ ]:
first_character_new = torch.cat((first_character,combined_tensor[12].unsqueeze(0)), dim=0)

In [ ]:
first_character_new.size()

In [ ]:
second_character = torch.cat((combined_tensor[5].unsqueeze(0),combined_tensor[3].unsqueeze(0)), dim=0)

In [ ]:
second_character.size()

In [ ]:
pcs_same_cha = pairwise_cosine_similarity(first_character_new, combined_tensor[4].unsqueeze(0))
pcs_same_cha


In [ ]:
pcs_two_cha = pairwise_cosine_similarity(combined_tensor[4].unsqueeze(0), second_character)
pcs_two_cha


In [ ]:
character_character_matching_threshold = 0.8

In [13]:
# Ассоциации между персонажами на основе заданного порога
char_i, char_j = torch.where(cos_matrix > character_character_matching_threshold)

NameError: name 'character_character_matching_threshold' is not defined

In [ ]:
character_character_associations = torch.stack([char_i, char_j], dim=1)

In [ ]:
# [0,0,0,1,-,1,-,-,0,-,1,0,1,1,1]

In [ ]:
import networkx as nx

g = nx.Graph(character_character_associations.tolist())

In [ ]:
C = [list(c) for c in nx.connected_components(g)]
C

In [ ]:
len(g.nodes)

In [ ]:
index_list = [0]*len(g.nodes)

In [ ]:
for i in range(len(C)):
    for j in C[i]:
        index_list[j] = i


In [ ]:
index_list

In [ ]:
nx.draw_spring(g, arrows=True, with_labels=True)

In [ ]:
adj_matrix = 

In [ ]:
character_character_associations

In [ ]:
character_ufds = UnionFind.from_adj_matrix(
    pcs
    > character_character_matching_threshold
)

In [ ]:
character_ufds.get_labels_for_connected_components()

In [ ]:
character_character_associations.size()

In [ ]:
plt.imshow(image_bboxes[0][4])

In [ ]:
crop_embeddings_for_batch[0].size()

### Список со всеми скорами

In [ ]:
exl = []
exl = np.array(exl)

In [ ]:
for tensor in character_scores:
    tensor_cpu = tensor.cpu()
    arrays = tensor_cpu.numpy()
    exl = np.concatenate((exl, arrays), axis=None)

### Датакласс

In [ ]:
@dataclass
class CropImage:
    image: np.array
    bbox: torch.Tensor
    embeddings: torch.Tensor
    character_score: float
    #page_number: int

In [ ]:
@dataclass
class Character:
    image_info: CropImage
    character_index: int

In [ ]:
import datasets

In [ ]:
# Создаем пустой список для хранения экземпляров класса CropImage
crop_images = []

In [ ]:
# Проходим по спискам данных и создаем экземпляры класса CropImage
for i, image_bbox in enumerate(image_bboxes):
    for j, bbox in enumerate(image_bbox):
        # Извлекаем соответствующие данные
        image = image_bbox[j]
        character_score = character_scores[i][j]
        embeddings = crop_embeddings_for_batch[i][j]
        bbox_tensor = crop_bboxes[i][j]
        
        # Создаем экземпляр класса CropImage и добавляем его в список
        crop_image = CropImage(image=image, bbox=bbox_tensor, embeddings=embeddings, character_score=character_score)
        crop_images.append(crop_image)

In [ ]:
type(crop_images)

In [ ]:
asdict(crop_images[0]).keys()

In [ ]:
schema = datasets.Features(
    {
        "image": datasets.Image(),
        "bbox": datasets.Sequence(feature=datasets.Value(dtype="float32")),
        "embeddings": datasets.Sequence(feature=datasets.Value(dtype="float32")),
        "character_score": datasets.Value(dtype="float32"),
    }
)

In [ ]:
ds = datasets.Dataset.from_list(list(map(asdict, crop_images)), features=schema)
ds.add_faiss_index("embeddings")
# ds.set_format("numpy")

In [ ]:
queries: NDArray[np.float64] = np.array(ds["embeddings"])
queries.shape

In [ ]:
rows = ds.num_rows

In [ ]:
from sklearn.preprocessing import normalize

np.linalg.norm(normalize(queries)[0])

In [ ]:
np.linalg.norm(queries[0])

In [ ]:
scores, examples = ds.get_nearest_examples_batch("embeddings", queries=np.array(ds["embeddings"]), k=rows)

In [ ]:
all_distances = []

for i in range(rows):
    all_distances.append(examples[i]["character_score"])

sc_array = np.array(all_distances)

In [ ]:
for score, sample, cha_score in zip(scores[0], examples[0]["image"], examples[0]["character_score"]):
    print(f"{score=}")
    print(f"{cha_score=}")
    sample.show()

In [ ]:
sc_array[0]

In [ ]:
dist_matrix = np.array(scores)
dist_matrix.shape

In [ ]:
dist_matrix.shape

In [ ]:
exl.shape

### Симметричная матрица

In [ ]:
change_distance = np.zeros((len(dist_matrix), len(dist_matrix)))

In [ ]:
for i in range(len(dist_matrix)):
    for j in range(len(dist_matrix)):
        cell_ind = np.where(exl == sc_array[i][j])
        cell_ind = cell_ind[0].item()
        change_distance[i][cell_ind] = dist_matrix[i][j]

---

In [ ]:
from scipy import sparse

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
dist_matrix.min()

In [ ]:
gaid = np.linalg.inv(diag)

---

In [ ]:
sns.heatmap(torch.from_numpy(dist_matrix))

In [ ]:
len(scores)

In [ ]:
qw = scores[0]

In [ ]:
zaq = scores[0]

In [ ]:
aa = np.std(qw)

In [ ]:
aa

In [ ]:
scores[0]

In [ ]:
scores

In [ ]:
for i in range(rows):
    scores, examples = ds.get_nearest_examples("embeddings", query=np.array(ds[i]["embeddings"]), k=rows)

In [ ]:
scores_one, examples_one = ds.get_nearest_examples("embeddings", query=np.array(ds[20]["embeddings"]), k=50)

In [ ]:
scores_one

In [ ]:
type(scores_one)

In [ ]:
new_score = scores/me

In [ ]:
new_score

In [ ]:
me = max(scores)

In [ ]:
me

In [ ]:
pp(scores)

In [ ]:
for score, el in zip(scores, examples["image"]):
    pp(score)
    el.show()

In [ ]:
pp(character_scores[0])

In [ ]:
pp(crop_bboxes)

In [ ]:
pp(crop_bboxes[0])

In [ ]:
pp(image_bboxes)

In [ ]:
a = torch.tensor([  99.3922,  244.3012,  454.1992,  477.5109])

In [ ]:
print(a)

In [ ]:
def array_to_image(array, tensor_key):
    """
    Преобразует np.array в изображение и сохраняет его по указанному пути, используя имя тензора в качестве имени файла.
    
    :param array: np.array, представляющий изображение.
    :param tensor_key: Тензор, используемый в качестве имени файла.
    """
    # Убедитесь, что массив имеет правильную форму

        
    # Преобразуем массив в PIL Image
    img = Image.fromarray(array)
    
    # Генерируем имя файла, используя имя тензора
    filename = f"crops_images/{tensor_key}.png"
    
    # Сохраняем изображение
    img.save(filename)


In [ ]:
type(image_bboxes)

In [ ]:
for i, dict_item in enumerate(image_bboxes):
    # Извлекаем тензор и np.array из словаря
    for tensor_key, array in dict_item.items():  # Используем.items() для перебора пар ключ-значение
        # Преобразуем np.array в изображение и сохраняем его, используя имя тензора в качестве имени файла
        array_to_image(array, tensor_key)

In [ ]:
len(image_bboxes)

In [ ]:
type(first_page)

In [ ]:
pp(image_bboxes)

In [ ]:
pp(crop_bboxes[0])

In [ ]:
pp(character_scores[0])

In [ ]:
pp(crop_embeddings_for_batch[0])

In [ ]:
image_bboxes[0].shape

In [ ]:
pp(image_bboxes[0].size())

In [ ]:
save_directory = 'crops_images'

In [ ]:
for i, array in enumerate(image_bboxes):
    # Сохранение изображения напрямую из массива
    Image.fromarray(array).save(f'{save_directory}/image_{i}.png')

In [ ]:
pp(character_scores)

In [ ]:
# with torch.no_grad():
#     results = model.predict_detections_and_associations(images)

In [ ]:
type(results[0])

In [ ]:
results[0].keys()

In [ ]:
bbox = BBox(
    coords=results[0]["characters"][0],
    score=results[0]["character_scores"][0],
    char_id=results[0]["character_cluster_labels"][0],
)
bbox

In [ ]:
page0 = Page(chars={0:[bbox1,bbox]}, shape=(1549, 1080), image=images[0])
page0

In [ ]:
pp(results[4])

Строим картинки на основании выхлопа модели

In [ ]:
now = time.localtime()
now_str = f"{now.tm_year}-{now.tm_mon:02}-{now.tm_mday:02}_{now.tm_hour:02}-{now.tm_min:02}-{now.tm_sec:02}"
out_dir = Path(f"out/{now_str}").resolve()
out_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
model.visualise_single_image_prediction(chapter, results[0], filename=f"{str(out_dir)}/chapter.png")

In [ ]:
# text_bboxes_for_all_images = [x["texts"] for x in results]

# for i in range(len(images)):
#     model.visualise_single_image_prediction(images[i], results[i], filename=f"{str(out_dir)}/image_{i}.png")